This is a notebbok for testing the BentoML service.
It demonstrates how to interact with the API endpoints using example requests and prints the responses.

Note that the server must be running locally at `http://localhost:3000` and MLflow serveice must be accessible.

In [1]:
import requests
from pydantic import BaseModel

In [2]:
BENTOML_URL = "http://localhost:3000"

In [3]:
class SpaceflightInput(BaseModel):
    engines: float
    passenger_capacity: int
    crew: float
    d_check_complete: bool
    moon_clearance_complete: bool
    iata_approved: bool
    company_rating: float
    review_scores_rating: float

class ModelURI(BaseModel):
    model_name: str
    model_version: int

# POST /import_model
### Import a model from MLflow Model Registry


### Request body:
```json
{
  "model_name": <string>,
  "model_version": <int>
}
```

### Response:
```json
{
  "message": "Model <model_name>/<model_version> is being imported and loaded."
}
```

In [4]:
def import_model(model_uri: ModelURI):
    url = f"{BENTOML_URL}/import_model"
    payload = {"model_uri": model_uri.model_dump()}
    response = requests.post(url, json=payload)
    return response.json()
    
model_uri = ModelURI(model_name="spaceflights-pandas", model_version=1)

res_import = import_model(model_uri)
print("Import model response:", res_import)
# Note: heavy computation, may take a while

Import model response: {'message': 'Model spaceflights-pandas/1 is being imported and loaded.'}


# POST /predict
### Predict using the loaded model

### Request body:    
```json
{
    "engines": 2,
    "passenger_capacity": 120,
    "crew": 8,
    "d_check_complete": true,
    "moon_clearance_complete": true,
    "iata_approved": true,
    "company_rating": 4.5,
    "review_scores_rating": 95.0
}
```
    
### Response:    
```json
{
    "prediction": [<prediction>]
}
```

In [7]:
def predict(input_data: SpaceflightInput):
    url = f"{BENTOML_URL}/predict"
    payload = {"input_data": input_data.model_dump()}
    response = requests.post(url, json=payload)

    try:
        return response.json()
    except Exception:
        print("Response non JSON:")
        print(response.status_code)
        print(response.text)
        return {"error": "Response is not valid JSON"}

example_input = SpaceflightInput(
    engines=2,
    passenger_capacity=4,
    crew=1,
    d_check_complete=True,
    moon_clearance_complete=True,
    iata_approved=True,
    company_rating=4.5,
    review_scores_rating=8.0
)

res_pred = predict(example_input)
print("Prediction response:", res_pred)

Prediction response: {'prediction': [2572.5585979712696]}
